<a href="https://colab.research.google.com/github/quangminh1212/Auto_Commit/blob/main/Trading_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile .env
API_KEY=eT0P26sdBoPcrIh6Bm45UsP41dWPW1ujh8EoeOmnjdiYH82UOsv7IKt8nS8RctVs
API_SECRET=ZvfthuxUVd3TgWdoUhYpRSF2AMNdr5QRVeJR4efiTsvjbUrZ7nHuAUVdNxY32jGL


Overwriting .env


In [2]:
!pip uninstall -y python-binance binance binance-connector
!pip uninstall -y binance python-binance binance-connector
!pip uninstall -y numpy pandas

!pip install numpy==1.26.0 pandas==2.2.2
!pip install --upgrade binance-connector


Found existing installation: binance-connector 3.12.0
Uninstalling binance-connector-3.12.0:
  Successfully uninstalled binance-connector-3.12.0
Found existing installation: numpy 1.26.0
Uninstalling numpy-1.26.0:
  Successfully uninstalled numpy-1.26.0
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
  Using cached numpy-1.26.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
  Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
Using cached numpy-1.26.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)
  Using cached binance_connector-3.12.0-py3-none-any.whl.metadata (13 kB)
Using cached binance_connector-3.12.0-py3-none-any.whl (74 kB)


In [3]:
!pip show binance-connector


Name: binance-connector
Version: 3.12.0
Summary: This is a lightweight library that works as a connector to Binance public API.
Home-page: https://github.com/binance/binance-connector-python
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: pycryptodome, requests, websocket-client
Required-by: 


In [5]:

from binance.um_futures import UMFutures

client = UMFutures(
    key="API_KEY",
    secret="API_SECRET",
    base_url="https://testnet.binancefuture.com"
)

print(client)


# from binance.um_futures import UMFutures

# # Thử khởi tạo client
# client = UMFutures("API_KEY", "API_SECRET", base_url="https://testnet.binancefuture.com")
# print(client)


ModuleNotFoundError: No module named 'binance.um_futures'

In [ ]:
import os
import time
import pandas as pd
import numpy as np

from dotenv import load_dotenv
load_dotenv()

from binance.um_futures import UMFutures

# Lấy API Key/Secret từ .env
API_KEY = os.getenv("API_KEY")
API_SECRET = os.getenv("API_SECRET")

# Binance USDM Futures Testnet base URL
BASE_URL = "https://testnet.binancefuture.com"

client = UMFutures(key=API_KEY, secret=API_SECRET, base_url=BASE_URL)

# Thông số Bot
SYMBOL = "BTCUSDT"
LEVERAGE = 20          # ví dụ high leverage
RISK_PER_TRADE = 0.02  # 2% vốn
CHECK_INTERVAL = 10    # lấy giá mỗi 10 giây
WINDOW = 10            # trung bình 10 giá gần nhất
BREAKOUT_RATIO = 0.001 # 0.1% break so với MA (tuỳ chỉnh)
STOP_LOSS_PCT = 0.005  # 0.5% stop-loss (tuỳ chỉnh)
TAKE_PROFIT_PCT = 0.01 # 1% take-profit (tuỳ chỉnh)

# Thiết lập leverage, margin type
def setup_leverage(symbol):
    try:
        client.change_leverage(symbol=symbol, leverage=LEVERAGE)
        client.change_margin_type(symbol=symbol, marginType="ISOLATED")
        print(f"[{symbol}] Đòn bẩy = {LEVERAGE}x, margin=ISOLATED")
    except Exception as e:
        print(f"Lỗi setup leverage: {e}")

setup_leverage(SYMBOL)

# Lưu các giá gần nhất (để tính MA). Lúc khởi động thì ta để trống.
price_history = []

def get_current_price(symbol):
    """
    Lấy giá markPrice hiện tại hoặc lastPrice. Ở đây dùng 'ticker_price' -> lastPrice.
    """
    data = client.ticker_price(symbol)
    return float(data["price"])

def get_usdt_balance():
    """
    Lấy số dư USDT
    """
    balances = client.balance()
    for b in balances:
        if b["asset"] == "USDT":
            return float(b["balance"])
    return 0.0

def get_position(symbol):
    """
    Kiểm tra xem đang có vị thế (LONG/SHORT) và khối lượng bao nhiêu
    """
    positions = client.position_information(symbol=symbol)
    # Mỗi cặp symbol trả về 2 dict (một cho "LONG", một cho "SHORT", nếu Hedge mode),
    #  hoặc 1 dict (one-way). Tuỳ setting account, ta duyệt check.
    for pos in positions:
        if pos["symbol"] == symbol:
            entry_price = float(pos["entryPrice"])
            position_amt = float(pos["positionAmt"])  # (+) = long, (-) = short, 0=không có
            return position_amt, entry_price
    return 0.0, 0.0

def close_position(symbol):
    """
    Đóng toàn bộ vị thế hiện tại (nếu có)
    => Gửi lệnh ngược lại quantity
    """
    position_amt, entry = get_position(symbol)
    if position_amt > 0:
        # Đang LONG => SELL hết
        qty = abs(position_amt)
        client.new_order(symbol=symbol, side="SELL", type="MARKET", quantity=qty, reduceOnly=True)
        print(f"[CLOSE LONG] {symbol} qty={qty}")
    elif position_amt < 0:
        # Đang SHORT => BUY hết
        qty = abs(position_amt)
        client.new_order(symbol=symbol, side="BUY", type="MARKET", quantity=qty, reduceOnly=True)
        print(f"[CLOSE SHORT] {symbol} qty={qty}")

def place_stop(symbol, stop_price, side):
    """
    Đặt lệnh stop-market đóng vị thế:
     - Nếu đang LONG => side=SELL
     - Nếu đang SHORT => side=BUY
    """
    try:
        client.new_order(
            symbol=symbol,
            side=side,
            type="STOP_MARKET",
            stopPrice=stop_price,
            closePosition=True
        )
        print(f"[STOP] {symbol} side={side} stopPrice={stop_price}")
    except Exception as e:
        print("Lỗi đặt stop:", e)

def open_position(symbol, side, quantity):
    """
    Mở vị thế MARKET. side = "BUY" => Long, side = "SELL" => Short
    """
    try:
        client.new_order(symbol=symbol, side=side, type="MARKET", quantity=quantity)
        print(f"[OPEN {side}] {symbol} qty={quantity}")
    except Exception as e:
        print("Lỗi mở vị thế:", e)

def run_bot():
    global price_history

    # 1) Lấy giá hiện tại
    current_price = get_current_price(SYMBOL)
    price_history.append(current_price)

    # 2) Chỉ giữ tối đa WINDOW giá gần nhất
    if len(price_history) > WINDOW:
        price_history.pop(0)

    # 3) Chưa đủ dữ liệu => bỏ qua
    if len(price_history) < WINDOW:
        print("Chưa đủ dữ liệu history => bỏ qua")
        return

    # 4) Tính MA
    ma = np.mean(price_history)

    # 5) Kiểm tra vị thế hiện tại
    position_amt, entry_price = get_position(SYMBOL)

    # 6) Tạo tín hiệu
    #    - LONG signal: current_price > ma + (ma * BREAKOUT_RATIO)
    #    - SHORT signal: current_price < ma - (ma * BREAKOUT_RATIO)
    upper_breakout = ma + ma * BREAKOUT_RATIO
    lower_breakout = ma - ma * BREAKOUT_RATIO

    # 7) Tính khối lượng => risk
    #    Lấy 2% vốn => * đòn bẩy => / current_price
    usdt_balance = get_usdt_balance()
    risk_fund = usdt_balance * RISK_PER_TRADE
    notional = risk_fund * LEVERAGE
    qty = notional / current_price
    qty = round(qty, 4)
    if qty < 0.001:
        qty = 0.001

    # 8) Nếu chưa có vị thế => tìm cơ hội mở
    if position_amt == 0:
        if current_price > upper_breakout:
            # Mở LONG
            open_position(SYMBOL, "BUY", qty)
            # Đặt stop-loss
            stop_loss_price = round(current_price * (1 - STOP_LOSS_PCT), 2)
            place_stop(SYMBOL, stop_loss_price, "SELL")
            # Đặt take-profit
            take_profit_price = round(current_price * (1 + TAKE_PROFIT_PCT), 2)
            try:
                client.new_order(
                    symbol=SYMBOL,
                    side="SELL",
                    type="TAKE_PROFIT_MARKET",
                    stopPrice=take_profit_price,
                    closePosition=True
                )
                print(f"[TP] {SYMBOL} SELL takeProfit={take_profit_price}")
            except Exception as e:
                print("Lỗi TP:", e)

        elif current_price < lower_breakout:
            # Mở SHORT
            open_position(SYMBOL, "SELL", qty)
            # Đặt stop-loss (ngược side = BUY)
            stop_loss_price = round(current_price * (1 + STOP_LOSS_PCT), 2)
            place_stop(SYMBOL, stop_loss_price, "BUY")
            # Đặt take-profit
            take_profit_price = round(current_price * (1 - TAKE_PROFIT_PCT), 2)
            try:
                client.new_order(
                    symbol=SYMBOL,
                    side="BUY",
                    type="TAKE_PROFIT_MARKET",
                    stopPrice=take_profit_price,
                    closePosition=True
                )
                print(f"[TP] {SYMBOL} BUY takeProfit={take_profit_price}")
            except Exception as e:
                print("Lỗi TP:", e)
        else:
            print("Không có tín hiệu breakout => đứng ngoài")

    # 9) Nếu đang có vị thế => có thể quản lý, trailing, v.v.
    else:
        # VD: Kiểm tra xem giá đã di chuyển ngược hướng quá mạnh => chủ động đóng
        # Ở đây ví dụ ta giao phó cho stop-loss, take-profit,
        # nên tạm thời kệ. Hoặc trailing stop...
        print(f"Đang giữ vị thế => amt={position_amt}, entry={entry_price}")

# --- Vòng lặp chính siêu ngắn hạn ---
try:
    while True:
        run_bot()
        time.sleep(CHECK_INTERVAL)  # 10 giây
except KeyboardInterrupt:
    print("Bot dừng.")
